<a href="https://colab.research.google.com/github/Yustira/crowd-counting/blob/master/Generate_density.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Density Map

This notebook is used to generate density map from image file saved on `images.npy` file for the purpose of crowd counting.
The density map is going to be saved as `images_density.npy` file

## Load the imports

In [2]:
import numpy as np
import scipy
from scipy import spatial
from scipy import ndimage
from tqdm import tqdm

## Load the image file

### Run this code only when you are using Colab
this code will download dataset from this [link](https://drive.google.com/file/d/1y0BoU7RIisMgj4PsCPbY02PPm4CEXaEF/view?usp=sharing) to colab's temporary storage

#### Install PyDrive

In [1]:
!pip install PyDrive

#### Google Drive Authentication

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#### Download the file from the link

In [4]:
IMAGE_FILE_ID = "1y0BoU7RIisMgj4PsCPbY02PPm4CEXaEF"
downloaded = drive.CreateFile({'id':IMAGE_FILE_ID})
downloaded.GetContentFile('images.npy')        

In [5]:
IMAGE_GT_FILE_ID = "1F8MSiZ974Hta6gZcex9DFvGwbhA2u3s8"
downloaded = drive.CreateFile({'id':IMAGE_GT_FILE_ID})
downloaded.GetContentFile('images_gt.npy') 

### Load dataset from file

In [6]:
images=np.load('images.npy')
images_gt=np.load('images_gt.npy',allow_pickle=True)

## Code to generate density map

In [7]:
def generate_density(X,Y):
    point=np.array([X,Y]).T
    leafsize=2048
    tree=spatial.KDTree(point.copy(),leafsize=leafsize)
    distances,locations=tree.query(point,k=4)
    density=np.zeros([480,640],dtype=np.float32)
    for i,pt in enumerate(point):
        pt2d=np.zeros([480,640],dtype=np.float32)
        pt=np.round(pt).astype('int')
        pt2d[pt[1],pt[0]]=1000
        sigma=min(max(1,int(np.sum(distances[i][1:3])*0.1)),8)
        density+=ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    return density

In [8]:
images_density=np.zeros([2000,480,640],dtype=np.float32)
for i in tqdm(range(2000)):
    images_density[i]=generate_density(images_gt[i][:,0],images_gt[i][:,1])

100%|██████████| 2000/2000 [19:39<00:00,  1.70it/s]


## Export the generated density map

In [11]:
np.save('images_density.npy',images_density)